# Resolving coreference with neuralcoref

In [18]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from tqdm import tqdm_notebook

There are few out-of-the-box libraries that support or specifically built for coreference resolution. Most wide-known are [CoreNLP](https://stanfordnlp.github.io/CoreNLP/coref.html), [Apache OpenNLP](https://opennlp.apache.org/) and [neuralcoref](https://github.com/huggingface/neuralcoref). In this short notebook, we will explore neuralcoref 3.0, a coreference resolution library by Huggingface.

First, let's install neuralcoref 3.0. To do this, we need to slightly downgrade spacy (neuralcoref is not compatible with the new cymem version used by the current version of spacy).

In [2]:
MODEL_URL = "https://github.com/huggingface/neuralcoref-models/releases/" \
            "download/en_coref_md-3.0.0/en_coref_md-3.0.0.tar.gz"

In [ ]:
#!pip install spacy==2.0.12

In [3]:
!pip install {MODEL_URL}

  ERROR: Command errored out with exit status 1:
   command: 'c:\users\programmer\appdata\local\programs\python\python37\python.exe' -u -c 'import io, os, sys, setuptools, tokenize; sys.argv[0] = '"'"'C:\\Users\\PROGRA~1\\AppData\\Local\\Temp\\pip-req-build-yr5uhnm9\\setup.py'"'"'; __file__='"'"'C:\\Users\\PROGRA~1\\AppData\\Local\\Temp\\pip-req-build-yr5uhnm9\\setup.py'"'"';f = getattr(tokenize, '"'"'open'"'"', open)(__file__) if os.path.exists(__file__) else io.StringIO('"'"'from setuptools import setup; setup()'"'"');code = f.read().replace('"'"'\r\n'"'"', '"'"'\n'"'"');f.close();exec(compile(code, __file__, '"'"'exec'"'"'))' bdist_wheel -d 'C:\Users\PROGRA~1\AppData\Local\Temp\pip-wheel-swjrdsw_'
       cwd: C:\Users\PROGRA~1\AppData\Local\Temp\pip-req-build-yr5uhnm9\
  Complete output (51 lines):
  running bdist_wheel
  running build
  running build_py
  creating build
  creating build\lib.win-amd64-3.7
  creating build\lib.win-amd64-3.7\en_coref_md
  copying en_coref_md\__init__.

  Running setup.py clean for en-coref-md
Failed to build en-coref-md
    Running setup.py install for en-coref-md: started
    Running setup.py install for en-coref-md: finished with status 'error'



  creating build\lib.win-amd64-3.7\en_coref_md\en_coref_md-3.0.0\neuralcoref
  copying en_coref_md\en_coref_md-3.0.0\neuralcoref\cfg -> build\lib.win-amd64-3.7\en_coref_md\en_coref_md-3.0.0\neuralcoref
  copying en_coref_md\en_coref_md-3.0.0\neuralcoref\pairs_model -> build\lib.win-amd64-3.7\en_coref_md\en_coref_md-3.0.0\neuralcoref
  copying en_coref_md\en_coref_md-3.0.0\neuralcoref\single_model -> build\lib.win-amd64-3.7\en_coref_md\en_coref_md-3.0.0\neuralcoref
  creating build\lib.win-amd64-3.7\en_coref_md\en_coref_md-3.0.0\neuralcoref\static_vectors
  copying en_coref_md\en_coref_md-3.0.0\neuralcoref\static_vectors\key2row -> build\lib.win-amd64-3.7\en_coref_md\en_coref_md-3.0.0\neuralcoref\static_vectors
  copying en_coref_md\en_coref_md-3.0.0\neuralcoref\static_vectors\vectors -> build\lib.win-amd64-3.7\en_coref_md\en_coref_md-3.0.0\neuralcoref\static_vectors
  creating build\lib.win-amd64-3.7\en_coref_md\en_coref_md-3.0.0\neuralcoref\tuned_vectors
  copying en_coref_md\en_core

In [4]:
!python -m spacy download en_core_web_md

You should consider upgrading via the 'C:\Users\Programmer\AppData\Local\Programs\Python\Python37\python.exe -m pip install --upgrade pip' command.



  Created wheel for en-core-web-md: filename=en_core_web_md-2.3.1-py3-none-any.whl size=50916645 sha256=a231b06e60c8d8f8e5036f0fe5dc991df05bdfe36fd715bfcaa00e7f3761a539
  Stored in directory: C:\Users\PROGRA~1\AppData\Local\Temp\pip-ephem-wheel-cache-8i_tzylk\wheels\43\1d\c1\a0af68d0648debf57f875e9dda56bbac35cfc27bfa187ffc46
Successfully built en-core-web-md
[+] Download and installation successful
You can now load the model via spacy.load('en_core_web_md')


## A small neuralcoref tutorial

How does this lib work? Let's find out!

First,we need to load the model:

In [6]:
import en_core_web_md
import neuralcoref
nlp =en_core_web_md.load()
#neuralcoref.add_to_pipe(nlp)
#coref = neuralcoref.NeuralCoref(nlp.vocab)
#nlp.add_pipe(coref, name='neuralcoref')



In [3]:
#token.set_extension('context', default=False, force=True)

NameError: name 'token' is not defined

Now we need a sentence with coreference. 

A boring theoretical reminder: coreference happens when* two different words denote the same entity* in the real world. In this competition, we deal with pronomial coreference. It comes in two flavors:
1. *Anaphora*, when a pronoun follows a noun: "John looked at me. He was clearly angry".
2. *Cataphora*, when it is vice versa: "When she opened the door, Jane realized that it was cold outside"

Let's start with two simple sentences with two anaphoric coreferences:

In [32]:
#test_sent = "The doctor came in. She held a paper in her hand."
test_sent = "Cook it on a barbeque till cooked."

Using neuralcoref is not really different from using plain spacy.

In [8]:
doc = nlp(test_sent)

To check if any kind of coreference was detected, `has_coref` attribute of the extension (referred to as `_`) is used:

In [28]:
doc._.has_coref

False

Great! We found something, let's see what exactly:

In [43]:
doc._.coref_clusters

You can go to the [website](https://huggingface.co/coref/?text=The%20doctor%20came%20in.%20She%20held%20a%20paper%20in%20her%20hand.) and play with the tool. It outputs cool resolution graphs like this one:

![graph](http://i66.tinypic.com/wtbmdi.png)

You can get the entity and coreferring pronouns from these clusters by simple indexing. The objects returned are in fact ordinary spacy `span`s.

In [44]:
doc._.coref_clusters

In [42]:
#doc._.coref_clusters[0].mentions

TypeError: 'NoneType' object is not subscriptable

## Deciding which entity the pronoun refers to

In competition data, the position of the entities and the pronoun comes as an offset from the beginning. Let's write a small function that will resolve coreference in a string and decide whether any of detected coreferring entities correspond to given offsets.


In [30]:
def is_inside(offset, span):
    return offset >= span[0] and offset <= span[1]

def is_a_mention_of(sent, pron_offset, entity_offset_a, entity_offset_b):
    doc = nlp(sent)
    if doc._.has_coref:
        for cluster in doc._.coref_clusters:
            main = cluster.main
            main_span = main.start_char, main.end_char
            mentions_spans = [(m.start_char, m.end_char) for m in cluster.mentions \
                              if (m.start_char, m.end_char) != main_span]
            if is_inside(entity_offset_a, main_span) and \
                    np.any([is_inside(pron_offset, s) for s in mentions_spans]):
                return "A"
            elif is_inside(entity_offset_b, main_span) and \
                    np.any([is_inside(pron_offset, s) for s in mentions_spans]):
                return "B"
            else:
                return "NEITHER"
    else:
        return "NEITHER"

A small test:

In [33]:
# "The doctor came in. She held a paper in her hand."
import neuralcoref 
entity_offset_a = test_sent.index("cook")
entity_offset_b = test_sent.index("barbeque")
pron_offset = test_sent.index("cooked")

is_a_mention_of(test_sent, pron_offset, entity_offset_a, entity_offset_b)

'NEITHER'

## Testing on the dataset 

In [34]:
gap_train = pd.read_csv("https://raw.githubusercontent.com/google-research-datasets/gap-coreference/master/gap-test.tsv", 
                       delimiter='\t', index_col="ID")

In [35]:
gap_train.head()

,Text,Pronoun,Pronoun-offset,A,A-offset,A-coref,B,B-offset,B-coref,URL
ID,,,,,,,,,,
test-1,Upon their acceptance into the Kontinental Hoc...,His,383,Bob Suter,352,False,Dehner,366,True,http://en.wikipedia.org/wiki/Jeremy_Dehner
test-2,"Between the years 1979-1981, River won four lo...",him,430,Alonso,353,True,Alfredo Di St*fano,390,False,http://en.wikipedia.org/wiki/Norberto_Alonso
test-3,Though his emigration from the country has aff...,He,312,Ali Aladhadh,256,True,Saddam,295,False,http://en.wikipedia.org/wiki/Aladhadh
test-4,"At the trial, Pisciotta said: ``Those who have...",his,526,Alliata,377,False,Pisciotta,536,True,http://en.wikipedia.org/wiki/Gaspare_Pisciotta
test-5,It is about a pair of United States Navy shore...,his,406,Eddie,421,True,Rock Reilly,559,False,http://en.wikipedia.org/wiki/Chasers


In [36]:
def predict(df):
    pred = pd.DataFrame(index=df.index, columns=["A", "B", "NEITHER"]).fillna(False)
    for i, row in tqdm_notebook(df.iterrows()):
        pred.at[i, is_a_mention_of(row["Text"], row["Pronoun-offset"], row["A-offset"], row["B-offset"])] = True
    return pred

In [37]:
train_preds = predict(gap_train)

c:\users\programmer\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:3: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  This is separate from the ipykernel package so we can avoid doing imports until


In [38]:
gap_train["NEITHER"] = np.logical_and(~gap_train["A-coref"], ~gap_train["B-coref"])

In [39]:
gap_train[["A-coref", "B-coref", "NEITHER"]].describe()

,A-coref,B-coref,NEITHER
count,2000,2000,2000
unique,2,2,2
top,False,False,False
freq,1082,1145,1773


In [40]:
train_preds.describe()

,A,B,NEITHER
count,2000,2000,2000
unique,1,1,1
top,False,False,True
freq,2000,2000,2000


In [41]:
from sklearn.metrics import classification_report
print(classification_report(gap_train[["A-coref", "B-coref", "NEITHER"]], train_preds[["A", "B", "NEITHER"]]))

              precision    recall  f1-score   support

           0       0.00      0.00      0.00       918
           1       0.00      0.00      0.00       855
           2       0.11      1.00      0.20       227

   micro avg       0.11      0.11      0.11      2000
   macro avg       0.04      0.33      0.07      2000
weighted avg       0.01      0.11      0.02      2000
 samples avg       0.11      0.11      0.11      2000



c:\users\programmer\appdata\local\programs\python\python37\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


We can see that though precision is quite good, we have very low recall. What can be done?
1. Remove excessive sentenes: if entities and the pronoun are contained in two sentences, we can strip other sentences.
2. Use neuralcoref's verdicts as a feature for another classifier (we would have to transform verdicts into probabilities anyway).